# Strictness and laziness
A function is called _non-strict_ if it may choose _not_ to evaluate one of its arguments. As an example, function `&&` takes two arguments but only evaluates the second argument if the first evaluates to `true`. This function could be called _strict in its first argument_ but _non-strict in the second_. Here's a non-strict `if2`:

In [33]:
def if2[A](cond: Boolean, onTrue: () => A, onFalse: () => A) =
    if (cond) onTrue() else onFalse()

val res = if2(false, 
            () => throw new Exception("Got true"), 
            () => "Got false")


defined function if2
res: String = "Got false"

Formally, a function `f` is strict if the expression `f(x)` evalutes to bottom (does not terminate) for all `x` that evaluate to `bottom`. An expression evaluates to `bottom` if it throws an error or runs forever instead of returning a definite value. 

The unevaluated form of an expression is called a _thunk_. Callers of the `if2` function explicitly need to create thunks, i.e., functions is empty argument list. Scala offers prettier syntax:

In [34]:
def if2[A](cond: Boolean, onTrue: => A, onFalse: => A): A = if(cond) onTrue else onFalse

if2(false, sys.error("Got exception"), "Got false") // Does not require `() => "Got false"`!

defined function if2
res33_1: String = "Got false"

The thunk is re-evaluated by default every time it's referenced:

In [3]:
def maybeTwice(b: Boolean, i: => Int): Int = if (b) i+i else 0

maybeTwice(true, { println("Evaluating"); 1+41 }) // Prints "Evaluating" twice!

Evaluating
Evaluating


defined function maybeTwice
res2_1: Int = 84

The result of the evaluation can be cached using the `lazy` keyword:

In [4]:
def maybeTwice2(b: Boolean, i: => Int): Int = {
    lazy val j = i // Caches the result, delays execution until `i` is first referenced
    if (b) j+j else 0
}
maybeTwice2(true, { println("Evaluating"); 1+41 }) // Prints "Evaluating" once!

Evaluating


defined function maybeTwice2
res3_1: Int = 84

### Lazy lists i.e. streams

In [5]:
sealed trait Stream[+A]
case object Empty extends Stream[Nothing]
case class Cons[+A](h: () => A, t: () => Stream[A]) extends Stream[A] // Constructor arguments cannot use `: =>` syntax

object Stream {
    // "Smart" constructor with nicer arguments and caching of evaluations
    def cons[A](hd: => A, tl: => Stream[A]): Stream[A] = {
        lazy val head = hd // Caches the result of evaluating `hd` so that it is only evaluated once
        lazy val tail = tl
        Cons(() => head, () => tail)
    }

    def empty[A]: Stream[A] = Empty // Helps type inference later on
    
    def apply[A](as: A*): Stream[A] = 
        if (as.isEmpty) Empty else cons(as.head, apply(as.tail: _*))
}

val testStream: () => Stream[Int] = () => Stream.cons(1, Stream.cons({ print("Evaluating second"); 2 }, Stream.cons({ sys.error("fail"); 3}, Stream.empty)))

defined trait Stream
defined object Empty
defined class Cons
defined object Stream
testStream: () => Stream[Int] = <function0>

Definitions are similar as for `List` (Chapter 3) except that the constructor `Cons` takes explicit _thunks_ that do not need to be evaluted. Extracting values from the stream requires forcing the evaluation:

In [6]:
def headOption[A](a: Stream[A]): Option[A] = 
    a match {
        case Empty => None
        case Cons(h, t) => Some(h()) // Forces evaluation
    }

// Using the smart constructor does not require explicit definition of thunks with `() => {}`
val stream = Stream.cons({ println("Evaluating..."); 3 }, Empty)

headOption(stream)
headOption(stream) // Does not print "Evaluating" as the value is cached in smart constructor

Evaluating...


defined function headOption
stream: Stream[Int] = Cons(<function0>,<function0>)
res5_2: Option[Int] = Some(3)
res5_3: Option[Int] = Some(3)

### Exercise 5.1
Write a function to convert a `Stream` to `List`.

In [7]:
def toList[A](a: Stream[A]): List[A] = {
    a match {
        case Empty => List()
        case Cons(h, t) => h() :: toList(t())
    }
}

toList(Stream(1, 2, 3))

defined function toList
res6_1: List[Int] = List(1, 2, 3)

### Exercise 5.2
Write the function `take(n)` for returning the first `n` elements of a `Stream`, and `drop(n)` for skipping the first `n` elements of a `Stream`.

In [8]:
def take[A](stream: Stream[A], n: Int): Stream[A] = {
    if (n <= 0) {
        Stream.empty
    } else {
        stream match {
            // This neither evaluates the expression in `h` nor `t` as `Stream.cons` defines its argument with `h: => A`
            // `Cons(h, t) => Cons(h, () => take(t(), n -1))` would also work
            case Cons(h, t) => Stream.cons(h(), take(t(), n - 1))
            case Empty => Stream.empty
        }
    }
}

def drop[A](stream: Stream[A], n: Int): Stream[A] = {
    if (n <= 0) {
        stream
    } else {
        stream match {
            case Cons(h, t) => drop(t(), n-1)
            case Empty => Stream.empty
        }
    }
}

val streamWithErrorAtHead = Stream.cons({ sys.error("fail") }, Stream.cons(2, Empty))
val streamWithErrorAtTail = Stream.cons(2, Stream.cons({ sys.error("fail") }, Empty))

toList(drop(streamWithErrorAtHead, 1)) // List(2)
toList(take(streamWithErrorAtTail, 1)) // List(2)

defined function take
defined function drop
streamWithErrorAtHead: Stream[Int] = Cons(<function0>,<function0>)
streamWithErrorAtTail: Stream[Int] = Cons(<function0>,<function0>)
res7_4: List[Int] = List(2)
res7_5: List[Int] = List(2)

### Exercise 5.3
Write the function `takeWhile`.

In [9]:
def takeWhile[A](stream: Stream[A], p: A => Boolean): Stream[A] = {
    stream match {
        case Cons(h, t) => if (p(h())) Stream.cons(h(), takeWhile(t(), p)) else Empty
        case Empty => Empty
    }
}

val stream = Stream.cons(1, Stream.cons({ print("Hi"); 2 }, Stream.cons({ sys.error("fail"); 3}, Stream.empty)))
val shortStream = takeWhile(stream, (n: Int) => n < 2) // Does not print anything
val evaluateStream = toList(shortStream) // Prints "Hi"

Hi

defined function takeWhile
stream: Stream[Int] = Cons(<function0>,<function0>)
shortStream: Stream[Int] = Cons(<function0>,<function0>)
evaluateStream: List[Int] = List(1)

Streams allow separating the concern of describing an expression from actually evaluating it. For example, _early termination_  is a breeze with streams:

In [10]:
def exists[A](stream: Stream[A], p: A => Boolean): Boolean = 
    stream match {
        case Cons(h, t) => p(h()) || exists(t(), p) // Does not touch the tail stream if finds match
        case Empty => false
    }

exists(testStream(), (a: Int) => a == 2)

Evaluating second

defined function exists
res9_1: Boolean = true

In [11]:
def foldRight[A,B](stream: Stream[A], z: => B)(f: (A, => B) => B): B = 
    stream match {
        case Cons(h, t) => f(h(), foldRight(t(), z)(f))
        case Empty => z
    }

foldRight(Stream(1, 2, 3), 0)(_ + _)

defined function foldRight
res10_1: Int = 6

In [12]:
def exists[A](stream: Stream[A], p: A => Boolean): Boolean = 
    foldRight(stream, false)((a, b) => p(a) || b)

val stream = Stream.cons(1, Stream.cons({ print("Evaluating second"); 2 }, Stream.cons({ sys.error("fail"); 3}, Stream.empty)))
exists(stream, (a: Int) => a == 2)

Evaluating second

defined function exists
stream: Stream[Int] = Cons(<function0>,<function0>)
res11_2: Boolean = true

### Exercise 5.4
Implement `forAll` that should terminate as soon as it encounters a nonmatching value.

In [13]:
def forAll[A](stream: Stream[A])(p: A => Boolean): Boolean = 
    foldRight(stream, true)((a, b) => p(a) && b)

forAll(testStream())(a => a < 2)

Evaluating second

defined function forAll
res12_1: Boolean = false

### Exercise 5.5
Use `foldRight` to implement `takeWhile`.

In [14]:
def takeWhile[A](stream: Stream[A])(p: A => Boolean): Stream[A] =
    foldRight(stream, Stream.empty: Stream[A])((a, b) => if (p(a)) Stream.cons(a, b) else Stream.empty)

toList(takeWhile(testStream())(a => a < 2))

Evaluating second

defined function takeWhile
res13_1: List[Int] = List(1)

### Exercise 5.7
Implement `map`, `filter`, `append`, and `flatMap` using `foldRight`.

To make things cleaner, let's re-define `Stream` and define the requested functions directly in the `Stream` trait. Note that this may mess things up in the Jupyter notebook if cells are executed in the "wrong" order.

In [16]:
sealed trait Stream[+A] {
    def foldRight[B](z: => B)(f: (A, => B) => B): B = 
        this match {
            case Cons(h, t) => f(h(), t().foldRight(z)(f))
            case Empty => z
        }
    
    def toList(): List[A] = {
        this.foldRight(List(): List[A])((a, b) => a :: b)
    }

    def exists(p: A => Boolean): Boolean =
        this.foldRight(false)((a, b) => p(a) || b)
    
    def map[B](f: A => B): Stream[B] =
        this.foldRight(Stream.empty: Stream[B])((a, b) => Stream.cons(f(a), b))
    
    def filter(p: A => Boolean): Stream[A] = 
        this.foldRight(Stream.empty: Stream[A])((a, b) => if (p(a)) Stream.cons(a, b) else b)
    
    // Cannot define `append(a: => A): Stream[A]` as `A` is a covariant type. It does not compile as it could cause 
    // funny situations, see 
    // https://stackoverflow.com/questions/43180310/covariant-type-a-occurs-in-contravariant-position-in-type-a-of-value-a
    def append[B >: A](b: => B): Stream[B] = 
        this.foldRight(Stream.cons(b, Stream.empty))((a, z) => Stream.cons(a, z))
    
    def flatMap[B](f: A => Stream[B]): Stream[B] =
        this.foldRight(Stream.empty: Stream[B])((a, z) => Stream.concatenate(f(a), z))

}
case object Empty extends Stream[Nothing]
case class Cons[+A](h: () => A, t: () => Stream[A]) extends Stream[A] // Constructor arguments cannot use `: =>` syntax

object Stream {
    // "Smart" constructor with nicer arguments and caching of evaluations
    def cons[A](hd: => A, tl: => Stream[A]): Stream[A] = {
        lazy val head = hd // Caches the result of evaluating `hd` so that it is only evaluated once
        lazy val tail = tl
        Cons(() => head, () => tail)
    }

    def empty[A]: Stream[A] = Empty // Helps type inference later on
    
    def apply[A](as: A*): Stream[A] = 
        if (as.isEmpty) Empty else cons(as.head, apply(as.tail: _*))
    
    def concatenate[A](a: Stream[A], b: Stream[A]): Stream[A] =
        a.foldRight(b)((a, z) => Stream.cons(a, z))
}

val testStream: () => Stream[Int] = () => Stream.cons(1, Stream.cons({ print("Evaluating second"); 2 }, Stream.cons({ sys.error("fail"); 3}, Stream.empty)))

defined trait Stream
defined object Empty
defined class Cons
defined object Stream
testStream: () => wrapper.wrapper.Stream[Int] = <function0>

In [21]:
Stream(1, 2, 3).map(_ * 2).toList()

Stream(1, 2, 3, 4).filter(_ % 2 == 0).toList()

Stream(1, 2, 3, 4).append({ println("Evaluating appended expression!"); 5 }).toList()

Stream.concatenate(Stream(1, 2, 3), Stream(4, 5, 6)).toList()

Stream(1, 2, 3, 4).flatMap(a => Stream(a, a)).toList()

Evaluating appended expression!


res20_0: List[Int] = List(2, 4, 6)
res20_1: List[Int] = List(2, 4)
res20_2: List[Int] = List(1, 2, 3, 4, 5)
res20_3: List[Int] = List(1, 2, 3, 4, 5, 6)
res20_4: List[Int] = List(1, 1, 2, 2, 3, 3, 4, 4)

Streams allow chaining operations without traversing the values multiple times. First consider an example with a list that requires traversing the list twice:

In [31]:
val addTen: Int => Int = a => {
    println(s"Adding ten to ${a}")
    a + 10
}

val isEven: Int => Boolean = a => {
    println(s"Checking if ${a} is divisible by two")
    a % 2 == 0
}

List(1, 2, 3, 4).map(addTen).filter(isEven)

Adding ten to 1
Adding ten to 2
Adding ten to 3
Adding ten to 4
Checking if 11 is divisible by two
Checking if 12 is divisible by two
Checking if 13 is divisible by two
Checking if 14 is divisible by two


addTen: Int => Int = <function1>
isEven: Int => Boolean = <function1>
res30_2: List[Int] = List(12, 14)

Now see how the order changes with `Stream` as the list is traversed only once:

In [32]:
Stream(1, 2, 3, 4).map(addTen).filter(isEven).toList()

Adding ten to 1
Checking if 11 is divisible by two
Adding ten to 2
Checking if 12 is divisible by two
Adding ten to 3
Checking if 13 is divisible by two
Adding ten to 4
Checking if 14 is divisible by two


res31: List[Int] = List(12, 14)